<a href="https://colab.research.google.com/github/MuhammedShiway/AICrowd-Mosquito/blob/main/%5B05%5D_Classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install ultralytics



---

# <center> PATHS </center>


---



In [2]:
# IMAGE PATHS
train_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Original /train_images'
test_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Original /test_images_phase1'

# CSV PATHS
train_csv_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Original /train.csv'
test_csv_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Original /test_phase1_v2.csv'

# YOLO PATHS
yolo_train = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Train'
yolo_test = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Test'

# MODEL PATHS
model_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Detection Models/RUN 7.pt'

# CROPPED PATHS
train_csv_crop = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Cropped/Train_CSV'
train_yolo_crop = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Cropped/Train_YOLO'
test_crop = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Cropped/Test'


csv_file = train_csv_path



---

# <center> IMPORT MODULES AND LIBRARIES </center>


---



In [3]:
import os
import random
import shutil
import csv
import torch
import pandas as pd
from tqdm import tqdm
from PIL import Image
%matplotlib inline
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.models as models
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, random_split




---

# <center> FOLDER CONTENT DELETER </center>


---



In [ ]:
def delete_folder_contents(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))






---

# <center> CSV FILE </center>


---





In [4]:
df = pd.read_csv(csv_file)

df.head()

,img_fName,img_w,img_h,bbx_xtl,bbx_ytl,bbx_xbr,bbx_ybr,class_label
0,92715872-3287-4bff-aa61-7047973e5c02.jpeg,2448,3264,1301,1546,1641,2096,albopictus
1,b0f7cc74-2272-4756-a387-38bcaf6965c8.jpeg,3024,4032,900,1897,1950,2990,albopictus
2,82df4b68-0f45-4afe-9215-48488bf3720e.jpeg,768,1024,220,58,659,808,albopictus
3,331ad30a-7564-4478-b863-7bc760adf5a1.jpeg,3456,4608,1169,2364,1586,2826,albopictus
4,1a46dbfb-104e-466b-88d7-98958d7b1fe9.jpeg,1024,1365,129,231,697,1007,culex


In [50]:
df['class_label'].unique()

array(['albopictus', 'culex', 'anopheles', 'culiseta',
       'japonicus/koreicus', 'aegypti'], dtype=object)

In [67]:
df['class_label'].value_counts()

albopictus            3567
culex                 3544
culiseta               492
japonicus/koreicus     321
anopheles               63
aegypti                 38
Name: class_label, dtype: int64



---

# <center> TRANSFORMS </center>


---



In [75]:
img_names = df['img_fName'].tolist()
general_labels = df['class_label'].tolist()

In [ ]:
general_labels

In [38]:
transform = transforms.Compose([
    transforms.Resize((384,384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [71]:
all_labels = ['albopictus', 'culex', 'anopheles', 'culiseta', 'japonicus/koreicus', 'aegypti']
label_to_int = {label: idx for idx, label in enumerate(all_labels)}

In [72]:
label_to_int

{'albopictus': 0,
 'culex': 1,
 'anopheles': 2,
 'culiseta': 3,
 'japonicus/koreicus': 4,
 'aegypti': 5}



---

# <center> CUSTOM DATALOADER </center>


---



In [53]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, img_names, labels, root_dir, transform=None):
        self.img_names = img_names
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.img_names[idx])
        image = Image.open(img_path)
        label_str = self.labels[idx]
        label = label_to_int[label_str]
        # this label returns the integer (0 to 5) corresponding to the species

        if self.transform:
            image = self.transform(image)

        return image, label


In [77]:
dataset = CustomDataset(img_names, general_labels, root_dir=train_csv_crop, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [78]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)



---

# <center> DEFINING / SELECTING MODEL </center>


---



In [80]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(set(general_labels)))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
pprint(device)

In [82]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [84]:
for inputs, labels in train_loader:
    print(type(inputs), type(labels))
    print(inputs.shape, labels.shape)
    break

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([32, 3, 384, 384]) torch.Size([32])




---

# <center> MODEL TRAINING </center>


---



In [87]:
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
                dataloader = train_loader
            else:
                model.eval()   
                dataloader = val_loader

            running_loss = 0.0
            correct_preds = 0

            pbar = tqdm(dataloader, desc=f"{phase} progress", unit="batch")

            for inputs, labels in pbar:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                correct_preds += torch.sum(preds == labels.data)

                pbar.set_postfix(loss=loss.item())

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = correct_preds.double() / len(dataloader.dataset)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
        print()

    return model


In [88]:
trained_model = train_model(model, criterion, optimizer, num_epochs=25)

Epoch 1/25
----------


train progress: 100%|██████████| 201/201 [14:44<00:00,  4.40s/batch, loss=0.169]


train Loss: 0.4519 Acc: 0.8621


val progress: 100%|██████████| 51/51 [05:48<00:00,  6.82s/batch, loss=0.0486]


val Loss: 0.3793 Acc: 0.8879

Epoch 2/25
----------


...

val Loss: 0.4233 Acc: 0.9153

Epoch 25/25
----------


train progress: 100%|██████████| 201/201 [02:17<00:00,  1.46batch/s, loss=0.00078]


train Loss: 0.0046 Acc: 0.9997


val progress: 100%|██████████| 51/51 [00:27<00:00,  1.87batch/s, loss=0.000478]

val Loss: 0.4312 Acc: 0.9165



In [94]:
torch.save(trained_model.state_dict(), '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Models/model_000.pth')



---

# <center> MODEL INFERENCE / TESTING </center>


---



In [90]:
test_df = pd.read_csv(test_csv_path)

test_df.head()

,img_fName,img_w,img_h
0,ce8f0ede-4366-45b9-bdd4-959e17c7618c.jpeg,1599,965
1,f207780c-0d0a-4d41-bd95-92ca53ff81a3.jpeg,822,861
2,99c6b7f9-dbea-4635-a18d-cde16b125515.jpeg,720,1280
3,662b4405-3600-41b9-9812-451b6d9df322.jpeg,3024,4032
4,1de14b6b-6781-4a24-ae5e-f760e865c758.jpeg,1242,2208


In [91]:
transform = transforms.Compose([
    transforms.Resize((384,384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


for index, row in test_df.iterrows():
    img_path = os.path.join(test_crop, row['img_fName'])
    image = Image.open(img_path)
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Predict
    with torch.no_grad():
        outputs = trained_model(image_tensor)
        _, predicted = torch.max(outputs, 1)
        class_label = all_labels[predicted.item()]

    # Save to CSV
    test_df.at[index, 'class_label'] = class_label


In [92]:
test_df

,img_fName,img_w,img_h,class_label
0,ce8f0ede-4366-45b9-bdd4-959e17c7618c.jpeg,1599,965,culex
1,f207780c-0d0a-4d41-bd95-92ca53ff81a3.jpeg,822,861,albopictus
2,99c6b7f9-dbea-4635-a18d-cde16b125515.jpeg,720,1280,albopictus
3,662b4405-3600-41b9-9812-451b6d9df322.jpeg,3024,4032,culex
4,1de14b6b-6781-4a24-ae5e-f760e865c758.jpeg,1242,2208,albopictus
...,...,...,...,...
2671,5071a8e8-b210-4929-bf48-7bafb4b8ba06.jpeg,3840,5120,culex
2672,fd610106-3aff-4172-9e12-5037d107af30.jpeg,4000,3000,albopictus
2673,c0677977-ef6f-4397-b809-a7f7c29e54f6.jpeg,4032,2268,culex
2674,df645ac5-dc5c-4549-95f2-7fbe1c4c4c90.jpeg,1078,1280,culex


In [93]:
# Save the updated TEST CSV
test_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Models/test_csv_000.csv', index=False)



---

# <center> ADD YOLO BBOX TO CSV  </center>


---



In [97]:
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path = model_path)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 178.4 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 10.1s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-7-31 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


In [100]:
def add_yolo_test_csv(image_folder, y_model):
    # Load the test_csv data
    df = test_df

    # For each image in the DataFrame
    for index, row in df.iterrows():

        # Perform inference on the image
        img_path = os.path.join(image_folder, row['img_fName'])
        results = y_model(img_path)

        # Get the bounding box info

        if len(results.xyxy[0]) > 0:  # If any objects were detected
            bbox_info = results.xyxy[0][0]

            # Convert the tensor values to integers
            xmin = round(bbox_info[0].item())
            ymin = round(bbox_info[1].item())
            xmax = round(bbox_info[2].item())
            ymax = round(bbox_info[3].item())

            # Add the bounding box information to the DataFrame
            df.loc[index, 'bbx_xtl'] = xmin
            df.loc[index, 'bbx_ytl'] = ymin
            df.loc[index, 'bbx_xbr'] = xmax
            df.loc[index, 'bbx_ybr'] = ymax
        else:  # No objects were detected
            print(f"No objects detected in image {row['img_fName']}")


            # For non-detected images, assign bounding box as the whole image
            img = Image.open(img_path)

            df.loc[index, 'bbx_xtl'] = 0
            df.loc[index, 'bbx_ytl'] = 0
            df.loc[index, 'bbx_xbr'] = img.width
            df.loc[index, 'bbx_ybr'] = img.height

    return df

In [103]:
test_yolo_csv = add_yolo_test_csv(test_path, yolo_model)

No objects detected in image 1049b2eb-9099-4d97-b9c8-fb2e3a8e1b82.jpeg
No objects detected in image 412c7c5e-a42a-47cb-a140-fcfea045e1fa.jpeg
No objects detected in image 976e23ac-cd82-449a-a1fb-ca426a0b1ce0.jpeg
No objects detected in image e24b8bc0-150c-4e67-9b2b-89cbe93d021a.jpeg
No objects detected in image 51f457f2-0616-40ff-bc68-9a4da3475635.jpeg
No objects detected in image e4dfc4c7-87cc-4c16-bc4a-5a4d0ab8f321.jpeg
No objects detected in image 877bf21e-b652-482e-9f7d-2f0bacbb9cea.jpeg
No objects detected in image 6e5c90db-f49d-4120-ad91-a0496225d1c8.jpeg
No objects detected in image 3f21c4ea-caca-45a6-8200-84e4f030d0e0.jpeg
No objects detected in image f5db356c-6e4a-4a5d-90f0-2a5e5aca5484.jpeg
No objects detected in image f18bef1f-7834-4dbd-9ecd-2bd50088500a.jpeg
No objects detected in image ffe9e181-d8ba-4ad2-9e9b-7740d5f0ae35.jpeg
No objects detected in image 573a5997-e70a-4c91-9acd-16958d1f09ea.jpeg
No objects detected in image d98d34a7-b83e-4bfa-ad2c-7bb7e2a5fa60.jpeg
No obj

In [107]:
test_yolo_csv

,img_fName,img_w,img_h,bbx_xtl,bbx_ytl,bbx_xbr,bbx_ybr,class_label
0,ce8f0ede-4366-45b9-bdd4-959e17c7618c.jpeg,1599,965,643.0,287.0,1003.0,641.0,culex
1,f207780c-0d0a-4d41-bd95-92ca53ff81a3.jpeg,822,861,85.0,213.0,591.0,670.0,albopictus
2,99c6b7f9-dbea-4635-a18d-cde16b125515.jpeg,720,1280,218.0,541.0,569.0,811.0,albopictus
3,662b4405-3600-41b9-9812-451b6d9df322.jpeg,3024,4032,1651.0,1612.0,2229.0,2256.0,culex
4,1de14b6b-6781-4a24-ae5e-f760e865c758.jpeg,1242,2208,422.0,753.0,823.0,1038.0,albopictus
...,...,...,...,...,...,...,...,...
2671,5071a8e8-b210-4929-bf48-7bafb4b8ba06.jpeg,3840,5120,1051.0,2236.0,3001.0,3530.0,culex
2672,fd610106-3aff-4172-9e12-5037d107af30.jpeg,4000,3000,458.0,690.0,2863.0,2278.0,albopictus
2673,c0677977-ef6f-4397-b809-a7f7c29e54f6.jpeg,4032,2268,1731.0,992.0,2077.0,1436.0,culex
2674,df645ac5-dc5c-4549-95f2-7fbe1c4c4c90.jpeg,1078,1280,325.0,355.0,738.0,877.0,culex


In [106]:
desired_order = ['img_fName', 'img_w', 'img_h', 'bbx_xtl', 'bbx_ytl', 'bbx_xbr', 'bbx_ybr', 'class_label']
test_yolo_csv = test_yolo_csv[desired_order]

In [108]:
    # Save the updated DataFrame as submission.csv
    test_yolo_csv.to_csv('/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Models/test_csv_000.csv', index=False)